In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import os
import nltk
from gensim import models
from gensim import similarities

In [ ]:
from collections import defaultdict
from gensim import corpora

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip3 install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 220 kB 29.0 MB/s 


In [ ]:
path = '/content/drive/MyDrive/NASA Space Apps Challenge/corpus/'
os.listdir(path)

['20090023312.pdf',
 '20090023312.json',
 '20090023596.pdf',
 '20090023596.json',
 '19930087094.pdf',
 '19930087094.json',
 '20010054945.pdf',
 '20010054945.json',
 '19980202494.pdf',
 '19980202494.json',
 '19990008606.pdf',
 '19990008606.json',
 '19990013870.pdf',
 '19990013870.json',
 '19970000423.pdf',
 '19970000423.json',
 '19970022109.pdf',
 '19970022109.json',
 '19960014856.pdf',
 '19960014856.json',
 '20050188450.pdf',
 '20050188450.json',
 '20080004223.pdf',
 '20080004223.json',
 '20080004359.pdf',
 '20080004359.json',
 '20080005959.pdf',
 '20080005959.json',
 '20010007235.pdf',
 '20010007235.json',
 '19980028457.pdf',
 '19980028457.json',
 '20080008254.pdf',
 '20080008254.json',
 '19960013911.pdf',
 '19960013911.json',
 '19960014854.pdf',
 '19960014854.json',
 '19960054114.pdf',
 '19960054114.json',
 '19960054110.pdf',
 '19960054110.json',
 '19960054124.pdf',
 '19960054124.json',
 '19970022873.pdf',
 '19970022873.json',
 '19970001432.pdf',
 '19970001432.json',
 '19970001434.pd

In [ ]:
import PyPDF2

In [ ]:
# print(filename)

In [ ]:
corpus_folder = '/content/drive/MyDrive/NASA Space Apps Challenge/corpus'

In [ ]:
import os, json
import pandas as pd
json_files = [pos_json for pos_json in os.listdir(corpus_folder) if pos_json.endswith('.json')]

file_ids = []
for f in json_files:
    file_id = f.split('.')[0]
    file_ids.append(file_id)

print(file_ids)

['20090023312', '20090023596', '19930087094', '20010054945', '19980202494', '19990008606', '19990013870', '19970000423', '19970022109', '19960014856', '20050188450', '20080004223', '20080004359', '20080005959', '20010007235', '19980028457', '20080008254', '19960013911', '19960014854', '19960054114', '19960054110', '19960054124', '19970022873', '19970001432', '19970001434', '19970022370', '19970022944', '19970019926', '20000085978', '20000091024', '20000102312', '20010066480', '20020039322', '19980169301', '19980197311', '19990004616', '19990004617', '19990014049', '19990027468', '19990046769', '19980017010', '19960029326', '20160011534', '19960047100', '19960051389', '20050179367', '19960020433', '19960041440', '19990079790', '20010000360', '19960047496', '19960048077', '19970006991', '19970001596', '19970001769', '19970010117', '19960017711', '20080012339', '19960015867', '19960020472', '19960042577', '19970001690', '19990019502', '19960022782', '19970005356', '19960044695', '20150018

In [ ]:
from pprint import pprint

with open('/content/drive/MyDrive/NASA Space Apps Challenge/broad_topics.json') as f:
    broad_topics = json.load(f)

pprint(broad_topics)

{'Engineering': {'Electronics and Electronic Engineering': ['amplifiers',
                                                            'audio amplifiers',
                                                            'batteries '
                                                            '(electrical '
                                                            'design)',
                                                            'bridge circuits',
                                                            'capacitors',
                                                            'cathode ray tubes '
                                                            '(electrical '
                                                            'design)',
                                                            'charge-coupled '
                                                            'devices',
                                                            'chips (integrated '
                 

In [ ]:
specific_categories = []
for topic in broad_topics.keys():
    specific_categories.extend(list(broad_topics[topic].keys()))

specific_categories

['Nuclear Physics',
 'Optics',
 'Electronics and Electronic Engineering',
 'Structural Mechanics',
 'Geophysics',
 'Energy Production and Conversion']

In [ ]:
documents = []
content = ''
for broad_cat in broad_topics:
    print(broad_cat)
    print("---"*10)
    for spec_cat in broad_topics[broad_cat]:
        print(spec_cat)
        documents = broad_topics[broad_cat][spec_cat]
        pprint(documents)

        stoplist = set('for a of the and to in'.split())
        texts = [
            [word for word in document.lower().split() if word not in stoplist]
            for document in documents
        ]

        # remove words that appear only once
        frequency = defaultdict(int)
        for text in texts:
            for token in text:
                frequency[token] += 1

        texts = [
            [token for token in text if frequency[token] >= 1]
            for text in texts
        ]

        dictionary = corpora.Dictionary(texts)
        corpus = [dictionary.doc2bow(text) for text in texts]

        lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=5)

        for file_id in file_ids:
            # print(file_name)
            with open(os.path.join(corpus_folder, file_id+'.json')) as f:
                json_file = json.load(f)

            if json_file['subjectCategory'] == spec_cat:
                file_name = os.path.join(corpus_folder, file_id+'.pdf')
                # print(file_name)
                print('\n')
                with open( file_name,'rb') as f:
                    f = PyPDF2.PdfFileReader(f)
                    doc = ''
                    for page in f.pages:
                        doc = doc + ' ' + page.extractText()

                vec_bow = dictionary.doc2bow(doc.lower().split())
                vec_lsi = lsi[vec_bow]  # convert the query to LSI space

                index = similarities.MatrixSimilarity(lsi[corpus])

                sims = index[vec_lsi]  # perform a similarity query against the corpus
                # print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples
                sims = sorted(enumerate(sims), key=lambda item: -item[1])
                keywords = []
                for doc_position, doc_score in sims[:5]:
                    print("{} : {}".format(documents[doc_position], round(float(doc_score), 4)))
                    keywords.append(documents[doc_position])
                    # print(documents[doc_position][:100], f": ({round(doc_score, 4)})")

                json_file['keywords'] = keywords

                with open(os.path.join(corpus_folder, file_id+'.json'), 'w') as f:
                    json.dump(json_file, f)


        # break
    print("\n")
    # break

Physics
------------------------------
Nuclear Physics
['alpha rays (theory)',
 'beta rays (theory)',
 'electron beams',
 'gamma rays (theory)',
 'ion beams (nuclear interactions)',
 'nuclear decay',
 'nuclear engines (theory)',
 'nuclear fission',
 'nuclear fuels',
 'nuclear fusion (theory)',
 'nuclear magnetic resonance',
 'nuclear particles',
 'nuclear physics',
 'nuclear power sources (theory)',
 'nuclear propulsion systems (theory)',
 'nuclear reactions',
 'nuclear reactor theory',
 'nuclear reactors',
 'nuclear research equipment',
 'nuclear scattering',
 'nuclear structure',
 'nuclear test equipment',
 'proton beams (nuclear interactions)',
 'radioisotopes',
 'reactor radiation safety measures (space applications)',
 'reactor theory',
 'space-power reactors (theory)',
 'x-ray radiation (theory)']




reactor theory : 0.9352
nuclear reactor theory : 0.85
reactor radiation safety measures (space applications) : 0.6589
nuclear decay : 0.204
nuclear fission : 0.204




reactor theory : 0.9308
nuclear reactor theory : 0.8712
reactor radiation safety measures (space applications) : 0.6437
nuclear decay : 0.2465
nuclear fission : 0.2465




nuclear decay : 0.9207
nuclear fission : 0.9207
nuclear fuels : 0.9207
nuclear particles : 0.9207
nuclear physics : 0.9207
Optics
['Bragg cells (optical applications)',
 'Cassegrain optics',
 'coherent light',
 'electron optics theory',
 'fiber optics',
 'geometrical optics',
 'infrared optics',
 'infrared radiation effects (optical applications)',
 'infrared signatures (optical applications)',
 'infrared spectra',
 'lens theory',
 'lenses (optical properties)',
 'light absorption',
 'light reflection',
 'light scattering',
 'light transmission',
 'liquid optics',
 'luminescence (optics)',
 'mirror interference (optics)',
 'modulation (optics)',
 'nonlinear optics',
 'optical bistability',
 'optical coatings',
 'optical communications (theory)',
 'optical fibers',
 'optical imaging devices (theory)',
 'optical materials',
 'optical properties',
 'optical waveguides',
 'optics',
 'optoelectronics (optics)',
 'photon beams',
 'photonics',
 'polarization (optics)',
 'refraction (optics)',

optical bistability : 0.9675
optical coatings : 0.9675
optical communications (theory) : 0.9675
optical fibers : 0.9675
optical imaging devices (theory) : 0.9675


optical bistability : 0.9407
optical coatings : 0.9407
optical communications (theory) : 0.9407
optical fibers : 0.9407
optical imaging devices (theory) : 0.9407




...

chemical energy conversion devices : 0.9501
geophysical energy conversion devices : 0.9501
hybrid energy conversion devices : 0.9501
photovoltaic energy conversion devices : 0.9501
thermionic energy conversion devices : 0.9501




solar generators : 0.9039
solar heating (space applications) : 0.8057
solar power (space applications) : 0.8057
solar panels (energy conversion) : 0.7038
solar cells (energy conversion) : 0.6553




magnetohydrodynamic (MHD) energy conversion device : 0.9344
energy production : 0.9304
energy storage : 0.9304
microwave energy transmission : 0.9127
alternative energy sources : 0.8942




magnetohydrodynamic (MHD) energy conversion device : 0.8857
energy production : 0.8645
energy storage : 0.8645
microwave energy transmission : 0.8455
alternative energy sources : 0.825




electric energy conversion devices : 0.8071
microwave energy conversion devices : 0.8053
chemical energy conversion devices : 0.7955
geophysical energy conversion devices : 0.7955
hybrid energy conversion devices : 0.7955


